The objective of this tutorial is to teach the basic usage of `DESTNOSIM` for simulations of discoverability of TNOs with DES.

We'll start with some standard imports and by creating the survey parameters:

In [ ]:
import numpy as np
import astropy.table as tb

If your `$DESDATA` path is set correctly, you can use the shortcut `y6a1c` to load all files related to the completeness estimates of the y6a1 release. If not, you can manually create it yourself using `Survey`. 

In [ ]:
import destnosim
y6 = destnosim.DES('y6a1c')

This object contains information for the completeness of all exposures, as well as their pointings. To make this easily accessible, we'll need to call the `createExposures()` method of `DES`. This makes the object indexable by exposure number, and returns a `DESExposure` object, that allows you to access the exposure's detection probability vs magnitude or the WCS. You probably won't need to access these yourself, though!

In [ ]:
y6.createExposures()
y6[228726]

Now, we need to figure out a population of objects to observe. I've included a file that contains the original [CFEPS-L7 model](https://iopscience.iop.org/article/10.3847/1538-3881/aa6aa5), except that the orbits were integrated (with the 4 giant planets as active particles + the Sun) to Jan. 1st, 2016. If you're interested in their model, see (and cite!) their paper. I've also assigned an `ORBITID` to each object, which I will use for bookkeeping purposes.

In [ ]:
cfeps = tb.Table.read('cfepsl7.fits')


Let's create a `Population` object. There are a few ways of doing this, depending on your input data. In this case, we have orbital elements (in a table format with a standard naming scheme), so we'll be creating a `ElementPopulation` object, which also has convenience functions for randomizing in an uniform manner the angles $\omega,\Omega,\mathcal{M}$. If you're trying to alter the other orbital elements, you'll need a `Distribution` object. More on these later!

If you have Cartesian (ICRS) phase space coordinates, you can create a `CartesianPopulation`. A subset of these are the Fibonacci sphere distributed objects, which you can access using `IsotropicPopulation`. 

These populations also require the epoch of the elements (in decimal years after J2000.0), so in our case, the epoch $t = 16.0 \, \mathrm{yr}$.

In [ ]:
cfeps_pop = destnosim.ElementPopulation(cfeps, 16.0)

Now, we need to project these orbits into the DES exposures. This will require a call to `orbitspp`, which is in C++. Make sure you have an `$ORBITSPP` environment variable that points to your installation of it!

We'll be calling the `createObservations` method of `DES`. This requires a `population`, as well as a string which will be the base for the files that will be created. This might take a while, depending on how big the population of objects is. For the CFEPS population, it might take up to an hour.

In [ ]:
y6.createObservations(cfeps_pop, 'CFEPS')

Now, we have the locations of all orbits in the DES exposures, that is, we know all potential detections of an object. To check whether these objects can actually be detected, we need to check their magnitudes against the completeness of each exposure. There are two factors to be considered:
 - the magnitude in a reference band of each object
 - the colors of each object
 
There are a few ways of doing this within `destnosim`. The `distribution` module provides a number of common distributions that you can sample from (eg a power law), or you might wish to use your own. 

CFEPS uses $g$ band magnitude as the reference, and the model already provides an *absolute* magnitude $H$ for each detection. The easiest possible construction, then, is to say each object has a delta-function distribution in their parent $H$. This is just a fancy way of saying that I'll be assigning the nominal $H$ coming from the CFEPS model to each object. 

In [ ]:
mags = [destnosim.DeltaFunction(i) for i in cfeps['H']]

mags[0].sample(1), mags[1].sample(10)

For the colors, in this case, we need to provide $g-r$, $g-i$ and $g-z$. In the Y6 paper, I constructed a nominal color distribution for each of the cold and hot Classical populations using the Y6 sample. For simplicity, here I will construct a synthetic color distribution that follows the nominal trend of the Y4 objects (see Figure 2 of the Y6 paper) and is uniformly distributed in $g-r$ between 0.4 and 1.5. These are implemented in the `magnitude` module.

In [ ]:
gr = destnosim.Uniform(0.4,1.5).sample(len(cfeps_pop))
gi, gz = destnosim.generate_colors(gr)
gr,gi,gz

Now, we go back to the population to assign the magnitude of each object. If you're using *absolute* magnitudes, you need the location of DECam at your reference epoch to obtain an apparent magnitude. For Jan 1st 2016, $\mathbf{x}_{\mathrm{DECam},\mathrm{ICRS}} = [-0.16147447,  0.89074144,  0.38593858] \, \mathrm{au}$. If you're using the *apparent* magnitude of the object, you can ignore this. You need, however, to specify the magnitude type when you assign the magnitudes!

If you wish to add a light curve to your object, right after running the magnitudes is where you'd implement this step. I won't here, for simplicity, but I will leave a few commented lines that explain how to do it.

In [ ]:
cfeps_pop.generateMagnitudes(mags, 'absolute', 'g', {'r':gr, 'i':gi,'z':gz}, 
                             [-0.16147447,  0.89074144,  0.38593858],
                             bands=['g','r','i','z'])

## Light curve implementation:
## lc = destnosim.RandomPhaseLightCurve(0.2) light curve with a random phase and a peak-to-peak amplitude of 0.2
## cfeps_pop.generateLightCurve(lc) applies the light curve to the objects
## If you wish to have a light curve for each object, make a list of light curves.

Now, we simulate the observations. This is done with the `des.DES` object, as the $m_{50}$ of each exposure are processing dependent.

In [ ]:
y6.observePopulation(cfeps_pop)

Finally, we go back to the `Population` and compute whether or not it would be detected. The criteria are as such:
 - $\mathtt{NUNIQUE} \geq 7$;
 - $\mathtt{ARCCUT} \geq 180$ days;
 - A triplet whose pairs are within 60 days (for objects with $d < 50$ au) or 90 days (for objects with $d > 50$ au) of each other
 
Since most of the CFEPS objects are closer than 50 au, I will use the 60 days threshold for all of them. After calling `computeStatistics()`, we get a table that compiles this information:

In [ ]:
cfeps_pop.computeStatistics(thresh=60)


In [ ]:
cfeps_pop.statistics

In [ ]:
def check_if_detected(population):
    '''
    Returns the ORBITIDs of each object that is detected according to the Y6 thresholds    
    '''
    st = population.statistics
    st['DETECTED'] = (st['TRIPLET']) & (st['NUNIQUE'] > 6) & (st['ARCCUT'] > 180)
    
    return st[st['DETECTED']]['ORBITID']

In [ ]:
check_if_detected(cfeps_pop)

And so we are done!